# EECS759P Coursework 2
- Name: Bheki Maenetja
- Student ID: 230382466

## Imports

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict, deque

## Plotting Functions

## Loading Data

In [ ]:
# Function provided in undirected_map.py
def load_data(df):
    station_dict = defaultdict(list)
    zone_dict = defaultdict(set)

    # get data row by row
    for index, row in df.iterrows():
        start_station = row[0]
        end_station = row[1]

        line = row[2] 
        
        act_cost = int(row[3])
        
        zone1 = row[4]
        zone2 = row[5]

        # station dictionary of child station tuples (child_name, cost from parent to the child)
        # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
        station_list = station_dict[start_station]
        station_list.append((end_station, line, act_cost))

        # the following two lines add the other direction of the tube "step"
        station_list = station_dict[end_station]
        station_list.append((start_station, line, act_cost))

        # we add the main zone
        zone_dict[start_station].add(zone1)
        # we add the secondary zone

        if zone2 != "0":
            zone_dict[start_station].add(zone2)
            # if the secondary zone is not 0 it's the main zone for the ending station
            zone_dict[end_station].add(zone2)
        else:
            # otherwise the main zone for the ending station is the same as for the starting station
            zone_dict[end_station].add(zone1)

    return station_dict, zone_dict

In [ ]:
tube_df = pd.read_csv('tubedata.csv', header=None)
tube_df.head()

In [ ]:
stations, zones = load_data(tube_df)

In [ ]:
set(tube_df[0])

In [ ]:
stations

In [ ]:
zones

## 2.0 Node Class

In [ ]:
class Node:
    def __init__(self, data, parent=None):
        self.data = data
        self.parent = parent

    def __str__(self):
        return f"{self.data[0]} via {self.data[1]} | Cost: {self.data[2]}"

    def __repr__(self):
        return self.__str__()

## 2.1 DFS, BFS and UCS

### Building optimal path to goal node

In [ ]:
def build_path(node):
    path_from_root = [node.data[0]]
    
    while node.parent:
        node = node.parent
        path_from_root = [node.data[0]] + path_from_root
    
    return path_from_root

### Depth-First Search

In [ ]:
def dfs_search(start, goal):
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    start_node = Node((start, None, None), None)
    
    frontier = [
        Node(s, start_node) 
        for s in stations[start].copy()
    ]
    
    explored = [start]
    num_explored = 0
    print(f"Start: {start} —>")
    
    while frontier:
        node = frontier.pop()
        num_explored += 1

        # Goal check
        if node.data[0] == goal:
            print(f"GOAL: {node.data[0]}")
            print(f"\nGoal found!\nNumber of explorations = {num_explored}")
            return node

        print(f"{node} ->")
        
        # Node expansion
        new_nodes = stations[node.data[0]].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

In [ ]:
# goal_node = dfs_search("Euston", "Victoria")
goal_node = dfs_search("Whitechapel", "Westminster")

In [ ]:
goal_node

In [ ]:
build_path(goal_node)

### Breadth-First Search

In [ ]:
def bfs_search(start, goal):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    start_node = Node((start, None, None), None)
    
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]
    
    explored = [start]
    num_explored = 0
    print(f"Start: {start} —>")
    
    while frontier:
        node = frontier.pop(0)
        num_explored += 1

        # Goal check
        if node.data[0] == goal:
            print(f"GOAL: {node.data[0]}")
            print(f"\nGoal found!\nNumber of explorations = {num_explored}")
            return node

        print(f"{node} ->")
        
        # Node expansion
        new_nodes = stations[node.data[0]].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

In [ ]:
# goal_node = bfs_search("Euston", "Victoria")
goal_node = bfs_search("Whitechapel", "Westminster")

In [ ]:
goal_node

In [ ]:
build_path(goal_node)

### Uniform Cost Search

In [ ]:
def ucs_search(start, goal):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    start_node = Node((start, None, None), None)
    
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]
    frontier.sort(key=lambda x: x.data[2])
    
    explored = [start]
    num_explored = 0

    print(f"START: {start} —>")
    
    while frontier:
        node = frontier.pop(0)
        num_explored += 1

        # Goal check
        if node.data[0] == goal:
            print(f"GOAL: {node.data[0]}")
            print(f"\nGoal found!\nNumber of explorations = {num_explored}")
            return node

        print(f"{node} ->")
        
        # Node expansion
        new_nodes = stations[node.data[0]].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

        frontier.sort(key=lambda x: x.data[2])

In [ ]:
goal_node = ucs_search("Euston", "Victoria")
# goal_node = ucs_search("Whitechapel", "Westminster")

In [ ]:
goal_node

In [ ]:
build_path(goal_node)

## 2.2 Comparison of DFS, BFS and UCS (report question)

## 2.3 Extending the Cost Function

In [ ]:
def ucs_search_2(start, goal, penalty=2):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    start_node = Node((start, None, None), None)
    
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]
    frontier.sort(key=lambda x: x.data[2])
    
    explored = [start]
    num_explored = 0

    # input(f"Starting frontier: {frontier} >>> ")
    print(f"START: {start} —>")
    
    while frontier:
        # input(f"\n\n===== NEW ITERATION =====\n\n")
        node = frontier.pop(0)
        num_explored += 1

        # input(f"Current node: {node} >>> ")
        
        # Goal check
        if node.data[0] == goal:
            print(f"GOAL: {node.data[0]}")
            print(f"\nGoal found!\nNumber of explorations = {num_explored}")
            return node

        print(f"{node} ->")
        
        # Node expansion
        # new_nodes = stations[node.data[0]].copy()
        new_nodes = [
            (c[0], c[1], c[2])
            if c[1] == node.data[1]
            else (c[0], c[1], c[2] + penalty) # applying penalty for line change
            for c in stations[node.data[0]].copy()
        ]
        # input(f"\nProspective children: {new_nodes} >>> ")
        # input(f"\nProspective children (with penalty applied): {new_nodes_2} >>> ")
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

        # input(f"\nNew frontier: {frontier} >>> ")
        frontier.sort(key=lambda x: x.data[2])
        # input(f"\nNew sorted frontier: {frontier} >>> ")

In [ ]:
# goal_node = ucs_search_2("Whitechapel", "Westminster")
goal_node = ucs_search_2("Euston", "Victoria")

In [ ]:
goal_node

In [ ]:
build_path(goal_node)